In [1]:
import json
from typing import Any, Dict, List

In [2]:
import re

with open('../../data/mlb/pbp/pbp_401356045.json', 'r', encoding='UTF8') as pbp_input:
    data = json.load(pbp_input)

In [3]:
teams = [
    data['away'],
    data['home']
]

def get_pitchers_first_appearance(data: dict) -> Dict[str, Dict[str, int]]:
    def get_pitching_team(atBat: str) -> str:
        return list(filter(lambda team: team != atBat, teams))[0]

    pitchers_first_appearance = {
        team: {}
        for team in teams
    }

    for period in data['periods']:
        pitching_team = get_pitching_team(period['atBat'])
        for event in period['events']:
            if 'type' in event and event['type'] == 'sub-p':
                pitcher = event['player']

                has_already_appeared = pitcher in pitchers_first_appearance[pitching_team]
                if not has_already_appeared:
                    pitchers_first_appearance[pitching_team][pitcher] = event['id']

    return pitchers_first_appearance

get_pitchers_first_appearance(data)

{'tex': {'C. Ragans': 5,
  'A.J. Alexy': 46,
  'B. Martin': 72,
  'J. Leclerc': 81,
  'M. Moore': 91},
 'min': {'S. Gray': 1, 'M. Fulmer': 63, 'G. Jax': 77, 'C. Thielbar': 86}}

In [8]:
after_pitch_events: Dict[int, Dict[str, Any]] = {}

for period in data['periods']:
    for event in period['events']:
        if 'type' in event and event['type'] == 'after-pitch':
            after_pitch_events[event['id']] = event

after_pitch_events

{9: {'isScoringPlay': False,
  'isInfoPlay': True,
  'score': {'away': 0, 'home': 0},
  'desc': 'Buxton stole second.',
  'type': 'after-pitch',
  'id': 9,
  'after-pitch-event': {'id': 10, 'pitch': 3}},
 39: {'isScoringPlay': False,
  'isInfoPlay': True,
  'score': {'away': 0, 'home': 1},
  'desc': 'Gordon caught stealing third, catcher to third.',
  'type': 'after-pitch',
  'id': 39,
  'after-pitch-event': {'id': 40, 'pitch': 1}},
 56: {'isScoringPlay': False,
  'isInfoPlay': True,
  'score': {'away': 2, 'home': 1},
  'desc': 'Seager to second on wild pitch by Gray, Duran to third on wild pitch by Gray.',
  'type': 'after-pitch',
  'id': 56,
  'after-pitch-event': {'id': 57, 'pitch': 1}}}

In [5]:
number_of_outs = {
    'caught stealing': 1,
    'strike looking': 1,
    'strike swinging': 1,
    'batters fielders choice runner out': 1,
    'line out': 1,
    'line out - double play': 2,
    'line out - triple play': 3,
    'ground out': 1,
    'ground out - double play': 2,
    'ground out - triple play': 3,
    'fly out': 1,
    'fly out - double play': 2,
    'fly out - triple play': 3,
    'foul out': 1,
    'foul out - double play': 2,
    'foul out - triple play': 3,
    'pop out': 1,
    'pop out - double play': 2,
    'pop out - triple play': 3,
}

for period in data['periods']:
    outs = 0
    bases = [0,0,0]
    print(period['atBat'])
    for event in period['events']:

        current_state_of_bases = bases.copy()

        description = event['desc']

        outcome_of_event = ''
        if 'pitches' in event:
            for pitch in  event['pitches'][:-1]:
                if pitch['result']['bases'] != current_state_of_bases:
                    current_state_of_bases = pitch['result']['bases'].copy()

            last_pitch = event['pitches'][-1]

            outcome_of_event = last_pitch['result']['outcome']

            match = re.search(r'((?:triple|double) play)', description)
            if match:
                outcome_of_event += f' - {match.group(1)}'

            bases = last_pitch['result']['bases']
        else:
            match = re.search(r'(caught stealing)', description)
            if match:
                outcome_of_event = 'caught stealing'

            match = re.search(r'(stole)', description)
            if match:
                outcome_of_event = 'steal'
                pass

        if outcome_of_event in number_of_outs:
            outs += number_of_outs[outcome_of_event]

        print(description, outcome_of_event, current_state_of_bases)

    if outs < 3:
        print('     - screwed up')
    print()

tex
S. Gray pitching for MIN  [0, 0, 0]
Semien lined out to center. line out [0, 0, 0]
Seager grounded out to first. ground out [0, 0, 0]
Lowe grounded out to second. ground out [0, 0, 0]

min
C. Ragans pitching for TEX  [0, 0, 0]
Arraez grounded out to shortstop. ground out [0, 0, 0]
Correa tripled to center. triple [0, 0, 0]
Buxton walked. ball [0, 0, 1]
Buxton stole second. steal [1, 0, 1]
Miranda struck out swinging. strike swinging [0, 1, 1]
Kepler grounded out to first. ground out [0, 1, 1]

tex
S. Gray pitching for MIN  [0, 0, 0]
García struck out swinging. strike swinging [0, 0, 0]
Heim struck out looking. strike looking [0, 0, 0]
Taveras grounded out to second. ground out [0, 0, 0]

min
C. Ragans pitching for TEX  [0, 0, 0]
Urshela flied out to right. fly out [0, 0, 0]
Gordon singled to left. single [0, 0, 0]
Beckham fouled out to right. foul out [1, 0, 0]
León popped out to second. pop out [1, 0, 0]

tex
S. Gray pitching for MIN  [0, 0, 0]
Calhoun struck out swinging. strike 